Her er jeres første rigtige opgave! I skal lave bruge [yfinance](https://mercantec.notion.site/yfinance-8f05decbf9a040659da38616e432ed62?pvs=4) også kaldet Yahoo Finance, til at trække data ned på jeres yndlingsaktie! I skal herfra lave en regressionsmodel på dataet. Det involvere at splitte dataet op i træningsdata og testdata. 
Nedenunder er der lige et code-snippet til at få jer startet!

Vi skal bruge følgende pakker til at starte vores projekt, en del af dem kender I. Pandas fra første dag, datetime har i nok arbejdet med lignende pakker før. [yfinance](https://mercantec.notion.site/yfinance-8f05decbf9a040659da38616e432ed62?pvs=4) er den som er linket til ovenfor og har alt det data vi skal bruge. Til sidste er der matplitlib, den er ikke strengt nødvendig for vores projekt, men det hjælper ofte at få et visuelt billede på! Dokumentation kan findes [her](https://mercantec.notion.site/MatPlotLib-bb10bec44c8e4bc28ac511017dbc895d?pvs=4)

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
from matplotlib import pyplot as plt

Vi bruger datetime pakken til at sætte et startspunkt og et slutpunkt. Startspunktet er for 365 dage siden og slutpunktet er for 2 dage siden!

In [ ]:
Start = date.today() - timedelta(365)
Start.strftime('%Y-%m-%d')

End = date.today() + timedelta(2)
End.strftime('%Y-%m-%d')

Vi laver en funktion som tager en "ticker" ind, altså en aktie. 
Her bruger vi så Pandas, pd, til at en dataframe som indeholder vores data omkring den valgte ticker, med slut og startsdato som definere før.

In [12]:
def closing_price(ticker):
    Asset = pd.DataFrame(yf.download(ticker, start=Start,
      end=End))     
    return Asset

Nu kan vi bruge vores funktion til at se på vores data. Her bruger vi Microsoft aktien, MSFT. Standarten er at vi for dato, åbningsprisen, højeste og laveste på dagen, lukke prisen og den justerede lukkepris. Vi kan så bruge matplotlib

In [ ]:
microsoft = closing_price('MSFT') 
print(microsoft)
plt.plot(microsoft['Adj Close'])
plt.show()

# Opgaven

Opgaven går nu på at I skal bruge jeres viden omkring regression på at lave en model som kan forudsige hvad aktie prisen bliver over den næste periode eller forhåbenligt bare en retning. 

Det er ikke målet at I skal finde en model som gør det, men at I skal lege rundt med det. De værktøjer som vi har fået indtil videre, er ikke nok til at kunne lave en model som har noget grundlag. Det data vi for er heller ikke nok. 

I kan tage inspiration ud fra dokumenterne her [regression](https://github.com/MAGS-Template/MachineLearning/blob/main/1.%20Regression/1.%20regression.ipynb) og [regression med data](https://github.com/MAGS-Template/MachineLearning/blob/main/1.%20Regression/2.%20regression_with_data.ipynb) - det er filerne i samme mappe som den her!


In [ ]:
import yfinance as yf
import pandas as pd
from datetime import date, timedelta
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt

In [ ]:
# Set the random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Function to get historical stock data
def closing_price(ticker, start, end):
    asset = pd.DataFrame(yf.download(ticker, start=start, end=end))
    return asset

# Function to create a dataset for training the model
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back):
        a = dataset[i:(i + look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

# Set start and end dates
start_date = date.today() - timedelta(365*3)
end_date = date.today()

# Get historical stock data
shopify = closing_price('SHOP', start=start_date, end=end_date)
# 1d array -> 2d array
dataset = shopify['Adj Close'].values.reshape(-1, 1)

# Normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1)) #finds the max values and min values
dataset = scaler.fit_transform(dataset) # scales the data

# Split the data into training and testing sets
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size, :], dataset[train_size:len(dataset), :]

# Create the dataset with look_back
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# Reshape input to be [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(trainX, trainY, epochs=50, batch_size=1, verbose=0)

# Make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# Invert predictions to original scale
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform([trainY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

# Plot training predictions
trainPredictPlot = np.empty_like(dataset)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(trainPredict) + look_back, :] = trainPredict

In [ ]:
# Plot testing predictions
testPredictPlot = np.empty_like(dataset)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(trainPredict) + (look_back * 2):len(dataset), :] = np.reshape(testPredict, (len(testPredict), 1))

# Plot the original and predicted prices
plt.plot(scaler.inverse_transform(dataset), label='Original Prices')
plt.plot(trainPredictPlot, label='Training Predictions')
plt.plot(testPredictPlot, label='Testing Predictions')
plt.legend()
plt.show()